# Вычислительная математика
## Численное интегрирование

## Задача численного интегрирования
Дана процедура вычисления значений $f(x)$. Найти приближенное значение определенного интеграла
$$
S(f) \approx I(f) = \int_a^b f(x) dx
$$

Основная идея:
* Приблизить $f$ *простой* функцией $\phi$, а функцию $\phi$ проинтегрировать аналитически

## Интерполяционные квадратурные формулы

* Рассмотрим отрезок $[-1,1]$, отобразим его на $[a,b]$:
$$
x = x(t) = \frac{a+b}{2} + \frac{b - a}{2} t
$$
Выберем различные узлы $t_1, \ldots, t_n \in [-1,1]$ и $x_i = x(t_i)$

* Построим интерполяционный многочлен:
$$
L_{n-1}(x) = \sum_{i=1}^n \prod_{j=1, \; j\ne i}^n f(x_i) \frac{x -x_j}{x_i - x_j}
$$

* Проинтегрируем по $[a,b]$
$$
S(f) = \int_a^b L_{n-1} (x) \, dx = \sum_{i=1}^n d_i \, f(x_i), \quad d_i = \frac{b-a}{2} \int_{-1}^1 \prod_{j=1,\;j\ne i}^n \frac{t - t_j}{t_i - t_j} \, dt
$$
* Формулы, полученные путем интегрирования интерполяционного многочлена называют формулами *Ньютона-Котеса*.

## Интерполяционные формулы, оценка ошибки

Если $f \in C^n[a,b]$, то из формулы для ошибки интерполяции получаем:
$$
\lvert I(f) - S(f) \rvert \le \frac{\lVert f^{(n)}\rVert_{C[a,b]}}{n!} \left( \frac{b-a}{2} \right)^{n+1} \int_{-1}^{+1} \bigg| \prod_{j=1}^n (t -t_j)\bigg| \, dt
$$

### Алгебраическая точность
* $S(f)$ имеет алгебраическую точность $m$, если она точна для многочленов степени $\le m$.  
* **Теорема:** Квадратурная формула $S$ с $n$ узлами имеет алгебраическую точность $m \ge n-1$ $\Leftrightarrow$ она является интерполяционной квадратурной формулой.

* $\Leftarrow$ По построению
* $\Rightarrow$ Подставим вместо $f$ многочлены Лагранжа $l_i(x)$, для которых формула точна.

## Примеры квадратурных формул

1) *Формула прямоугольников с центральной точкой*: $S(f) = f(\frac{a+b}{2}) h$,  ($h = b - a$)  
Интерполяция 0-го порядка. Оценка ошибки для $f \in C^1$: $\lvert I(f) - S(f)\rvert \le \frac{1}{4}M_1 h^2$.  
Ту же формулу можно получить из линейной интерполяции Эрмита для кратного узла $t_1 = t_2 = 0$: $H(t) = f(0) + f'(0) t$. Следовательно, можно улучшить оценку погрешности для $f \in C^2$:
$$
\lvert I(f) - S(f) \rvert \le \frac{1}{24} M_2 h^3
$$



2) *Формула трапеций*: $S(f) = \frac{1}{2} (f(a) + f(b)) h$. Для $f \in C^2$
$$
\lvert I(f) - S(f)\rvert \le \frac{1}{12} M_2 h^3
$$

3) *Формула Симпсона*: обычная интерполяция по 3-м точкам $-1,0,+1$ и эрмитова интерполяция с кратным узлом $-1,0,0,+1$ дают один и тот же результат:
$$
S(f) = \frac{h}{6} \left(f(a) + f(\frac{a+b}{2}) + f(b)\right)
$$

## Составные квадратурные формулы
